In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import sklearn.linear_model as lm
PATH = 'data'

In [3]:
df = pd.read_pickle('{}/log_returns_cleaned.pkl'.format(PATH))
# Remove stocks that have less than 99% clean data
df = df.dropna(thresh=len(df)*.99, axis=1)
# Remove days with NAs
df = df.dropna(axis=0)

In [4]:
# Remove ETFs
spy = df['SPY']
df = df.drop('SPY', axis=1)
df = df.drop(['SPY', 'IWM', 'EEM', 'TLT', 'USO', 'GLD', 'XLF', 'XLB', 'XLK', 'XLV', 'XLI', 'XLU', 'XLY', 'XLP', 'XLE'], axis=1, errors='ignore')

We compute the excess return for each stock by $S_t - \beta M_t$, where $M_t$ is the market return (here the SPY) and $\beta$ is fit by OLS. There is a small amount of lookahead bias here (technically this should be computed via a rolling regression), but the difference in the final values is negligible.

In [5]:
pred = pd.DataFrame(lm.LinearRegression(fit_intercept=False).fit(spy.values.reshape(-1,1), df.values).predict(spy.values.reshape(-1, 1)), 
                    index=df.index, columns=df.columns.values)
df_excess = df - pred

Now, to avoid liquidity/size bias, lend realism to the frictionless trading assumption and for Marcenko-Pastur reasons we restrict to only the 500 most liquid stocks. 

In [6]:
df_500_volume = pd.read_pickle('{}/cleaned_multi.pkl'.format(PATH))
df_500_volume = df_500_volume['volume'].loc[:,df.columns.values,:].unstack(1).mean().sort_values()[-500:]

df_500 = df.loc[:, df_500_volume.index.values]
df_exc_500 = df_excess.loc[:, df_500_volume.index.values]
df_exc_500

ticker,MTB,WRI,PVH,AVB,SLG,TTI,ARW,RJF,EV,ELY,...,ORCL,EBAY,F,GE,CSCO,INTC,MSFT,BAC,AAPL,IRM
date,,,,,,,,,,,,,,,,,,,,,
2000-03-29,0.016073,-0.001092,0.050644,0.011097,0.016745,0.008833,-0.063188,0.001553,0.006298,-0.008979,...,-0.049340,-0.118784,0.005798,0.042740,-0.024887,-0.029800,0.026096,-0.031223,-0.024312,0.008528
2000-03-30,0.002411,0.022952,0.009393,0.012279,-0.001725,0.094649,-0.022055,0.012811,0.039405,0.001538,...,-0.029736,0.060917,0.059240,-0.007566,-0.010733,-0.016437,-0.018381,0.003313,-0.059375,0.008659
2000-03-31,-0.004412,-0.011925,0.029579,-0.003653,-0.019664,0.011745,0.029158,0.023048,0.030100,-0.011725,...,-0.018672,-0.176811,-0.027952,-0.032486,0.034272,0.023948,0.015508,0.013820,0.064595,0.002806
2000-04-03,0.037012,0.000649,0.010001,-0.012249,-0.002161,-0.008582,-0.052704,-0.023861,-0.022927,-0.006028,...,-0.022465,-0.213076,0.016133,0.027475,-0.065758,-0.017295,-0.162440,0.047682,-0.024946,-0.015319
2000-04-04,-0.000972,0.019644,-0.016406,0.010378,-0.003639,-0.003075,0.054817,-0.051935,-0.056567,-0.000341,...,-0.003088,0.162646,0.051612,-0.036114,0.012226,0.025526,-0.017888,-0.012250,-0.037859,-0.005727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-05-17,0.005678,0.005127,-0.008575,0.007908,0.011859,-0.052776,-0.026229,0.007926,-0.009895,-0.013502,...,0.005411,-0.001947,-0.003106,-0.005672,0.015872,-0.005999,0.000161,0.003482,0.001420,-0.003629
2019-05-20,0.017427,-0.005862,-0.017671,-0.006658,-0.000661,0.024555,0.012627,0.016340,-0.004139,-0.034196,...,-0.007374,-0.001043,0.006727,-0.004663,0.002530,-0.021732,-0.007541,0.010381,-0.024489,-0.009896
2019-05-21,-0.004471,0.004079,0.013449,-0.000109,0.001691,-0.003567,0.009809,0.001124,0.060385,-0.020708,...,-0.001579,-0.010565,-0.014315,-0.001960,-0.002547,0.009163,-0.004110,-0.003885,0.009140,0.000361


In [8]:
df_500.to_pickle('{}/top_500_returns.pkl'.format(PATH))
df_exc_500.to_pickle('{}/top_500_excess_returns.pkl'.format(PATH))
df_excess.to_pickle('{}/excess_returns_clean.pkl'.format(PATH))
df.to_pickle('{}/returns_clean.pkl'.format(PATH))
spy.to_pickle('{}/spy_lr.pkl'.format(PATH))